In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### 모델 로드

In [2]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

In [3]:
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-summarization')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/682k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

#### 요약문 생성

In [4]:
from torch.utils.data import DataLoader
import pandas as pd

In [5]:
# 테스트 데이터셋 로드
df = pd.read_csv("/content/drive/MyDrive/응용자연어처리/project/data/short_test.csv")

In [6]:
# 배치 요약문 생성 함수
def generate_batch_summaries(model, tokenizer, texts, device, batch_size=16):
    dataloader = DataLoader(texts, batch_size=batch_size)
    summaries = []
    for batch in dataloader:
        # 텍스트를 배치로 토큰화
        inputs = tokenizer(
            list(batch),
            return_tensors="pt",
            max_length=1024,
            truncation=True,
            padding="max_length"
        ).to(device)

        # 모델로 요약 생성
        outputs = model.generate(
            inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=128,
            num_beams=4,
            early_stopping=True
        )

        # 요약문 디코딩
        batch_summaries = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        summaries.extend(batch_summaries)
    return summaries

# GPU/CPU 설정
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# 배치 처리 및 요약문 생성
batch_size = 16
df["generated_summary"] = generate_batch_summaries(
    model, tokenizer, df["contents"].tolist(), device, batch_size
)

In [7]:
df.head()

,contents,summary,char_length,length,generated_summary
0,자유한국당이 ‘2020년 회계연도 예산안 100대 문제 사업’에 5·18 진상규명관...,자유한국당이 ‘2020년 회계연도 예산안 100대 문제 사업’에 5·18 진상규명관...,770,short,광주광역시의회 김동찬 의장은 성명을 통해 “5·18 진상규명 관련 사업예산은 진상규...
1,의정부시는 다음 달 1일부터 10월 31일까지 ‘제7회 의정부시 사진 공모전’을 개...,"의정부시는 다음 달 1일부터 10월 31일까지 '일상생활 속 쉼표, 아름다운 의정부...",770,short,의정부시는 다음 달 1일부터 10월 31일까지 ‘제7회 의정부시 사진 공모전’을 개...
2,국산차로는 최초로 인증받은 대체부품(인증품)이 이달 14일에 본격 출시함에 따라 자...,11일 국토교통부에 따르면 국내 자동차 부품회사인 창원금속공업(주)은 국산차로는 최...,789,short,국산차로는 최초로 인증받은 대체부품(인증품)이 이달 14일에 본격 출시함에 따라 자...
3,박영재 인천항만공사는(사장 남봉현)는 재기창업·재도전기업 지원사업에 최종 선발된 2...,인천항만공사는 재기창업·재도전기업 지원사업을 실시하여 2개의 기업을 선정하고 1년간...,789,short,인천항만공사는 재기창업·재도전기업 지원사업에 최종 선발된 2개 기업과 인천대 창업지...
4,편집에디터 광주·전남지방중소벤처기업청(청장 김문환)는 1일 2019년 제2차 광주?...,광주·전남지방중소벤처기업청은 1일 2019년 제2차 전남지역 백년가게로 선정된 업체...,882,short,광주·전남지방중소벤처기업청은 1일 2019년 제2차 광주? 전남지역 백년가게로 선정...


In [8]:
summaries = df["summary"]
generated_summaries = df["generated_summary"]

#### 성능평가 - short

ROUGE: https://github.com/HeegyuKim/korouge

In [9]:
!pip install korouge_score

In [10]:
import numpy as np
from korouge_score import rouge_scorer

In [11]:
# 한국어 ROUGE
scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL", "rougeLsum"])

In [12]:
# ROUGE 점수 계산
rouge_scores = []
for reference, generated in zip(df["summary"], df["generated_summary"]):
    scores = scorer.score(reference, generated)
    rouge_scores.append(scores)

# 결과 확인
for i, scores in enumerate(rouge_scores[:5]):
    print(f"Sample {i + 1}:")
    print(scores)
    print()

Sample 1:
{'rouge1': Score(precision=0.25, recall=0.13043478260869565, fmeasure=0.1714285714285714), 'rouge2': Score(precision=0.13043478260869565, recall=0.06666666666666667, fmeasure=0.08823529411764705), 'rougeL': Score(precision=0.20833333333333334, recall=0.10869565217391304, fmeasure=0.14285714285714285), 'rougeLsum': Score(precision=0.20833333333333334, recall=0.10869565217391304, fmeasure=0.14285714285714285)}

Sample 2:
{'rouge1': Score(precision=0.8461538461538461, recall=0.36666666666666664, fmeasure=0.5116279069767441), 'rouge2': Score(precision=0.6666666666666666, recall=0.27586206896551724, fmeasure=0.3902439024390244), 'rougeL': Score(precision=0.8461538461538461, recall=0.36666666666666664, fmeasure=0.5116279069767441), 'rougeLsum': Score(precision=0.8461538461538461, recall=0.36666666666666664, fmeasure=0.5116279069767441)}

Sample 3:
{'rouge1': Score(precision=0.2, recall=0.11538461538461539, fmeasure=0.14634146341463417), 'rouge2': Score(precision=0.07142857142857142

In [13]:
# ROUGE 점수 평균 계산 (fmeasure 평균)
metrics = ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']
average_scores = {metric: 0 for metric in metrics}

# 총 샘플 수
n = len(rouge_scores)

# 각 메트릭의 fmeasure 합산
for score in rouge_scores:
    for metric in metrics:
        average_scores[metric] += getattr(score[metric], "fmeasure", 0)

# 각 메트릭의 평균 계산
for metric in metrics:
    average_scores[metric] /= n

print(average_scores)

{'rouge1': 0.29870364438457997, 'rouge2': 0.18226607014003993, 'rougeL': 0.2808317127941718, 'rougeLsum': 0.28092232268756806}


BERT-Score: https://github.com/Tiiiger/bert_score

In [14]:
import torch
from transformers import BertTokenizer, BertModel
from scipy.spatial.distance import cosine

In [15]:
# BERT-Score
bert_tokenizer = BertTokenizer.from_pretrained("beomi/kcbert-base")
bert_model = BertModel.from_pretrained("beomi/kcbert-base").eval()

def get_bert_embeddings(sentence, tokenizer, model):
    # 문장을 토큰화하고 BERT 입력 형식으로 변환
    inputs = tokenizer(sentence, return_tensors='pt', padding=True, truncation=True)

    # BERT 모델을 통해 문장의 임베딩 얻기
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = torch.mean(outputs.last_hidden_state, dim=1).squeeze(0)

    return embeddings.numpy()

def calculate_bert_score(sentence1, sentence2, tokenizer, model):
    # 각 문장의 BERT 임베딩 벡터 얻기
    embedding1 = get_bert_embeddings(sentence1, tokenizer, model)
    embedding2 = get_bert_embeddings(sentence2, tokenizer, model)

    # 코사인 유사도 계산 (1 - cosine distance)
    score = 1 - cosine(embedding1, embedding2)

    return score

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/250k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [16]:
# BERT-Score 점수 계산
df["bert_score"] = df.apply(lambda row: calculate_bert_score(row["summary"], row["generated_summary"], bert_tokenizer, bert_model), axis=1)
print(df.head())

                                            contents  \
0  자유한국당이 ‘2020년 회계연도 예산안 100대 문제 사업’에 5·18 진상규명관...   
1  의정부시는 다음 달 1일부터 10월 31일까지 ‘제7회 의정부시 사진 공모전’을 개...   
2  국산차로는 최초로 인증받은 대체부품(인증품)이 이달 14일에 본격 출시함에 따라 자...   
3  박영재 인천항만공사는(사장 남봉현)는 재기창업·재도전기업 지원사업에 최종 선발된 2...   
4  편집에디터 광주·전남지방중소벤처기업청(청장 김문환)는 1일 2019년 제2차 광주?...   

                                             summary  char_length length  \
0  자유한국당이 ‘2020년 회계연도 예산안 100대 문제 사업’에 5·18 진상규명관...          770  short   
1  의정부시는 다음 달 1일부터 10월 31일까지 '일상생활 속 쉼표, 아름다운 의정부...          770  short   
2  11일 국토교통부에 따르면 국내 자동차 부품회사인 창원금속공업(주)은 국산차로는 최...          789  short   
3  인천항만공사는 재기창업·재도전기업 지원사업을 실시하여 2개의 기업을 선정하고 1년간...          789  short   
4  광주·전남지방중소벤처기업청은 1일 2019년 제2차 전남지역 백년가게로 선정된 업체...          882  short   

                                   generated_summary  bert_score  
0  광주광역시의회 김동찬 의장은 성명을 통해 “5·18 진상규명 관련 사업예산은 진상규...    0.902772  
1  의정부시는 다음 달 1일부터 10월 31일까지 ‘제7회 의정부시 사진 공모전’을 개...    0.907185  
2  국산

In [17]:
# BERT-Score 평균
df["bert_score"].mean()

0.8515265352742323